PRM-KClosest - Basic Approach
======================

Version | Author
------------ | -------------
0.2 | Björn Hein

License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)

This work-sheet should include all necessary incredience to implement different PRM algorithm for path planning applications in Python.


Important links are:

    General Info: http://www.python.org
    Python tutorial http://www.python.org/doc/tut/
    NetworkX http://networkx.github.io/
    NumPy and SciPy Guide http://docs.scipy.org/
    Matplotlib gallery http://matplotlib.sourceforge.net/gallery.html

Remember that:

    you have to press ctrl-return or shift-return, to execute the code in the code sections, only then the variables are "generated" and can be used
    you can execute the whole notebook by Cell->runAll






Setting up a basic PRM-Algorithm
===============================

Needed functionality
--------------------

* Random position generator (Sampling strategy)
* Computation of **nearest neighbours** for a given node
* Test of connected components
* Collision test (point and **line**): Please take care, we also need a **line test**.



PRM Basic
===========

Following a simplified version of the classical PRM:

* no sophisticated way of searching nearest neighbours (just brute-force) AND don't care about the order
* no sophisticated way to detect complicated areas

Remarks:

* discussed in the lecture different nearestNeighbour-function were added or changed
* also the IKPerfMonitor-decorator is used to quantify the needed time, etc...


Definition of a base class for PRMs
======================================

see file IPPlanerBase.py:

```
from IKPlanerBase import PlanerBase
class PRMBase(PlanerBase):
    
    def __init__(self,_collChecker):
        super(PRMBase,self).__init__(_collChecker)

    def _getRandomPosition(self):
        limits = self.collChecker.getWorldLimits()        
        pos = [random.uniform(limit[0],limit[1]) for limit in limits]
        return pos
    
    def _getRandomFreePosition(self):
        pos = self._getRandomPosition()
        while self.collChecker.isInCollision(pos):
            pos = self._getRandomPosition()
        return pos
```

Based on the base class mentioned before, the PRM algorithm can be implemented in the following way:

In [ ]:
%matplotlib inline

In [ ]:
import IPPRMBase
from IPPerfMonitor import IPPerfMonitor
import networkx as nx
import random
import numpy as np
import math
import heapq

# reduce coding effort by using function provided by scipy
from scipy.spatial.distance import euclidean, cityblock

# sort nearest neighbour (KDTree)
from scipy.spatial import KDTree


class KClosestPRM(IPPRMBase.PRMBase):

    def __init__(self, _collChecker):
        super(KClosestPRM, self).__init__(_collChecker)
        self.graph = nx.Graph()

    
    @IPPerfMonitor
    def _inSameConnectedComponent(self, node1, node2):
        """ Check whether to nodes are part of the same connected component using
            functionality from NetworkX
        """
        for connectedComponent in nx.connected_components(self.graph):
            if (node1 in connectedComponent) & (node2 in connectedComponent):
                return True

        return False


    @IPPerfMonitor
    def _nearestNeighboursK(self, pos, k):
        """ Brute Force method to find all nodes of a 
        graph near the given position **pos** with in the distance of
        **radius**
        Remark: This function was added during the lecture
        """

        heap = list()
        for node in self.graph.nodes(data=True):
            heapq.heappush(heap, (euclidean(node[1]['pos'],pos), node))

        # sort in ascending order
        result = list()
        while (len(heap) > 0) and (len(result) < k) :
             result.append(heapq.heappop(heap)[1]) 

        return result
    
    @IPPerfMonitor
    def euclidean2(self, p, q):
        """ Computes the squared euclidean distance (2D) between the two given points.
        Use this as a more efficient way to compare distances between two points.
        Remark: This function was added during the lecture
        """
        return (p[0] - q[0]) ** 2 + (p[1] - q[1]) ** 2

    @IPPerfMonitor
    def _nearestNeighboursKV2(self, pos, k):
        """ Remark: This function was added during the lecture
        """
        sortKey = lambda data: euclidean(pos, data[1]["pos"])
        nodeList = sorted(list(self.graph.nodes(data=True)), key=sortKey)
        # return smallest k elements
        return nodeList

    
    @IPPerfMonitor
    def _learnRoadmapNearestNeighbourKDTree(self, k, numNodes):
        """ Very very unsafe: assumes certain order of nodes (TODO: to be made explicit)"""
        # set up number of collision free nodes
        for i in range(numNodes):
            self.graph.add_node(i, pos=self._getRandomFreePosition())

        # for every node in graph find nearest neigbhours
        posList = list(nx.get_node_attributes(self.graph,'pos').values())
        kdTree = KDTree(posList)
        
        for node in self.graph.nodes(data=True):
        # Find set of candidates to connect to sorted by distance
            result = kdTree.query(node[1]['pos'],k)

            # check connection
            for data in result[1]:
                if not self._inSameConnectedComponent(node[0],data):
                #if True:
                    if not self._collisionChecker.lineInCollision(node[1]['pos'],posList[data]):
                         self.graph.add_edge(node[0],data)
                    else:
                        continue
    
    @IPPerfMonitor
    def _learnRoadmapNearestNeighbour(self, k, numNodes):
        """ Generate a roadmap by given number of nodes and radius, that should be tested for connection."""
        # nodeID is used for uniquely enumerating all nodes and as their name
        nodeID = 1
        while nodeID <= numNodes:
        
            # Generate a 'randomly chosen, free configuration'
            newNodePos = self._getRandomFreePosition()
            self.graph.add_node(nodeID, pos=newNodePos)
            
            # Find set of candidates to connect to sorted by distance
            result = self._nearestNeighboursKV2(newNodePos, k)
            
            # for all nearest neighbours check whether a connection is possible
            counter = 0
            for data in result:
                if self._inSameConnectedComponent(nodeID,data[0]):
                    continue
                
                if not self._collisionChecker.lineInCollision(newNodePos,data[1]['pos']):
                    self.graph.add_edge(nodeID,data[0])
                    
                counter += 1
                if counter == k+1:
                    break
            
            nodeID += 1
    
    @IPPerfMonitor
    def planPath(self, startList, goalList, config):
        """
        
        Args:
            start (array): start position in planning space
            goal (array) : goal position in planning space
            config (dict): dictionary with the needed information about the configuration options
            
        Example:
        
            config["k"]   = 5
            config["numNodes"] = 300
            config["useKDTree"] = True
            
            startList = [[1,1]]
            goalList  = [[10,1]]
            
            instance.planPath(startList,goalList,config)
        
        """
        # 0. reset
        self.graph.clear()
        
        # 1. check start and goal whether collision free (s. BaseClass)
        checkedStartList, checkedGoalList = self._checkStartGoal(startList,goalList)
        
        # 2. learn Roadmap
        #self._learnRoadmapNearestNeighbourKDTree(config["k"],config["numNodes"])
        self._learnRoadmapNearestNeighbour(config["k"],config["numNodes"])

        # 3. find connection of start and goal to roadmap
        # find nearest, collision-free connection between node on graph and start
        result = self._nearestNeighboursK(checkedStartList[0], config["k"])
        for node in result:
            if not self._collisionChecker.lineInCollision(checkedStartList[0],node[1]['pos']):
                 self.graph.add_node("start", pos=checkedStartList[0], color='lightgreen')
                 self.graph.add_edge("start", node[0])
                 break

        result = self._nearestNeighboursK(checkedGoalList[0], config["k"])
        for node in result:
            if not self._collisionChecker.lineInCollision(checkedGoalList[0],node[1]['pos']):
                 self.graph.add_node("goal", pos=checkedGoalList[0], color='lightgreen')
                 self.graph.add_edge("goal", node[0])
                 break

        try:
            path = nx.shortest_path(self.graph,"start","goal")
        except:
            return []
        return path

## Visualizing the information

In [ ]:
import matplotlib.pyplot as plt
def basicPRMVisualize(graph, collChecker, solution, ax = None, nodeSize = 300):
    """ Draw graph, obstacles and solution in a axis environment of matplotib.
    """
    # get a list of positions of all nodes by returning the content of the attribute 'pos'
    pos = nx.get_node_attributes(graph,'pos')
  
    # get a list of degrees of all nodes
    # degree = nx.degree_centrality(graph)
    
    # draw graph (nodes colorized by degree)
    nx.draw(graph, pos, ax = ax, node_size=nodeSize)
    
    # draw all connected components, emphasize the largest one
    Gcc=(graph.subgraph(c) for c in nx.connected_components(graph))
    G0=next(Gcc) # [0] = largest connected component
    
    # how largest connected component
    nx.draw_networkx_edges(G0,pos,
                               edge_color='y',
                               width=nodeSize/100.0, ax = ax
                            )
    
    nx.draw_networkx_labels(G0,pos,
                               font_size=nodeSize/20.0, ax = ax
                            )
    # show other connected components
    for Gi in Gcc:
       if len(Gi)>1:
          nx.draw_networkx_edges(Gi,pos,
                                 edge_color='grey',
                                 alpha=0.3,
                                 width=nodeSize/150.0, ax = ax
                                 )
    

    collChecker.drawObstacles(ax)
    
    
    # draw nodes based on solution path
    Gsp = nx.subgraph(graph,solution)
    nx.draw_networkx_nodes(Gsp,pos,
                            node_size=nodeSize,
                             node_color='g',  ax = ax)
        
    # draw edges based on solution path
    nx.draw_networkx_edges(Gsp,pos,alpha=0.8,edge_color='g',width=nodeSize/30.0,  ax = ax)
        
    # draw start and goal
    for startGoal in ["start","goal"]:
        if startGoal in graph.nodes(): 
            nx.draw_networkx_nodes(graph,pos,nodelist=[startGoal],
                                       node_size=nodeSize,
                                       node_color='#00dd00',  ax = ax)
        

Simple testing of the algorithm 
==================================

In [ ]:
from shapely.geometry import Point, Polygon, LineString
from IPEnvironment import CollisionChecker

# Definition of a testing environment
trapField = dict()
trapField["obs1"] = LineString([(6, 18), (6, 8), (16, 8), (16,18)]).buffer(1.0)

environment = CollisionChecker(trapField)

basicPRM = KClosestPRM(environment)
basicConfig = dict()
basicConfig["k"] = 5
basicConfig["numNodes"] = 300

start = [[10,10]]
goal  = [[10,1]]

IPPerfMonitor.clearData()

solution = basicPRM.planPath(start, goal, basicConfig)

In [ ]:
fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)

basicPRMVisualize(basicPRM.graph,environment,solution,ax=ax,nodeSize=100)


Playground
===========

Check the results from, when using different strategies to build up the roadmap. Comment/Uncomment corresponding parts in the Code

* _learnRoadmapNearestNeighbourKDTree(config["k"],config["numNodes"])
* _learnRoadmapNearestNeighbour(config["k"],config["numNodes"])
